# Homework 4 - Genetic Networks #

## Negative Feedback Loop - Stress Response ##

Consider Stress molecules (S) that are controlled by Enzymes (E) which are controlled by Repressors (R). The concentrations of these components are regulated by the following equations:

$$ \frac{dR}{dt} = \frac{\gamma}{1+(\frac{R}{k})^h} - \frac{R}{\tau_R} + l $$
$$ \frac{dS}{dt} = \Theta_t - \gamma E S $$
$$ \frac{dE}{dt} = \frac{1}{1+(\frac{R}{k})^h} - \frac{E}{\tau_E} $$

Initial conditions: $ S(0) = E(0) = 0, R(0) = 1 $.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

In [ ]:
# initial conditions
R0 = 0
S0 = 0
E0 = 0

par = [0.2,1,1,0,1,2]

# we define Theta_t(t), which is the stress source, to be a step function that activates and spikes at a certain 
# point in time. This makes sense since the stress is usually caused by an abrupt change or signal

def Theta_t(t):
    return np.where(t > 0, 1, 0)

# define the equations
def dr(R, t, par):
    # par[0] = gamma
    # par[1] = k
    # par[2] = h
    # par[3] = l
    # par[4] = tau_R
    # par[5] = tau_E
    
    drdt = par[0] / (1 + (R/par[1])**par[2]) - (R/par[4]) + par[3]
    return drdt

def ds(S, E, t, Theta_t, par):
    
    dsdt = Theta_t(t - 10)*(1-Theta_t(t-30)) - par[0] * E * S       
    print(t)
    print(t-10)
    print(Theta_t(t-10))
    print(dsdt)
    return dsdt

def de(E, R, t, par):
    
    dedt = 1 / (1+(R/par[1])**par[2]) - E / par[5]
    return dedt

# set the time space
t_s = np.linspace(0,50,5000)
print(t_s)

# Solve 
R = odeint(dr, R0, t_s, args=(par,))
E = odeint(de, E0, t_s, args=(R,par))
S = odeint(ds, S0, t_s, args=(E, Theta_t, par))
plt.plot(R, t_s, c='r', label='R(t)')
plt.plot(E, t_s, c='g', label='E(t)')
plt.plot(S, t_s, c='b', label='S(t)')
plt.legend()
plt.grid()
plt.xlabel('t')
plt.ylabel('Population')
plt.title('Stress response')
plt.show()

[0.00000000e+00 1.00020004e-02 2.00040008e-02 ... 4.99799960e+01
 4.99899980e+01 5.00000000e+01]
[[0.        ]
 [0.00992744]
 [0.01970749]
 ...
 [0.04090794]
 [0.04089955]
 [0.04089117]]
[[-10.        ]
 [ -9.99007256]
 [ -9.98029251]
 ...
 [ -9.95909206]
 [ -9.95910045]
 [ -9.95910883]]
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


RuntimeError: The array return by func must be one-dimensional, but got ndim=2.